***firstTestNotebook.ipynb
Marcus Viscardi,    February 21, 2022***

# First test of using jupiter notebooks inside pycharm
## Hopefully all goes well?

After a day or so of use: It's definitely nice to be able to iterate so quickly with small parts of the code, but juggling some issues that come with overwriting variables in annoying. I imagine this will be useful for initial exploratory steps of writing some code, while in the longer run having solid scripts will make outputs more deterministic.


### Imports:

In [32]:
import pandas as pd
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)
import seaborn as sea
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

from nanoporePipelineCommon import load_and_merge_lib_parquets
print("Imports complete.")

Imports complete.


### Load all three library sets into one long-format dataframe:

In [4]:
libs_to_run = ['polyA', 'polyA2', 'polyA3', 'totalRNA2', 'totalRNA3']
reads_df_raw, compressed_df_raw = load_and_merge_lib_parquets(libs_to_run,
                                                              drop_sub_n=1,
                                                              add_tail_groupings=False,
                                                              pass_list_columns=True)
compressed_df_raw['lib'] = compressed_df_raw['lib'].replace({'polyA':'polyA1'})
print('Done.')

Loading readAssignments file from: /data16/marcus/genomes/elegansRelease100/Caenorhabditis_elegans.WBcel235.100.allChrs.parquet... Done.
Looking for files for libraries: ['polyA', 'polyA2', 'polyA3', 'totalRNA2', 'totalRNA3']
Looking for file for totalRNA2, at /data16/marcus/working/210720_nanoporeRun_totalRNA_0639_L3_replicate/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for polyA, at /data16/marcus/working/210528_NanoporeRun_0639_L3s/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for polyA2, at /data16/marcus/working/210719_nanoporeRun_polyA_0639_L3_replicate/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for polyA3, at /data16/marcus/working/220131_nanoporeRun_polyA_0639_L3_third/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for totalRNA3, at /data16/marcus/working/220131_nanoporeRun_totalRNA_0639_L3_third/output_dir/merge_files/*_mergedOnReads.parquet..

100%|██████████| 65002/65002 [00:01<00:00, 39173.41it/s]


Gene counts pre sub-1 gene_hits drop:  65002
Gene counts post sub-1 gene_hits drop:  65002
Done.


### Create a seperated copy of the loaded long-format dataframe. Drop any genes that have less than XX reads in any libs:

In [5]:
min_hits_all_libs = 40

compressed_df = compressed_df_raw.copy()
compressed_df = compressed_df.query(f"gene_hits >= {min_hits_all_libs}")
# Crazy merge call just to show a df which confirms that compressed_df_raw isn't changed!!
# print(pd.merge(compressed_df[['lib', 'gene_id', 'gene_hits']],
#                compressed_df_raw[['lib', 'gene_id', 'gene_hits']],
#                suffixes=('_filtered', '_raw'), 
#                how='outer', on=['lib', 'gene_id'])
#       .sort_values('gene_id').head())

### Add a method to turn the long format dataframe into a wide format, allowing for operations lib calculating tail length or read count differences:
This will probably be useful for other things, I should move this to the common methods script

In [6]:
def multilib_long_to_wide_df(long_df,
                             transpose_col='lib',
                             remerge_cols=('gene_id',
                                           'gene_name',
                                           'chr_id')):
    unique_transpose_values = long_df[transpose_col].unique()
    df_holder_dict = {}
    for unique_val in unique_transpose_values:
        df_holder_dict[unique_val] = long_df.query(f"{transpose_col} == '{unique_val}'").drop(columns=[transpose_col])
    print(f"First, merging {unique_transpose_values[0]} & {unique_transpose_values[1]}")
    wide_df = pd.merge(df_holder_dict[unique_transpose_values[0]],
                       df_holder_dict[unique_transpose_values[1]],
                       on=remerge_cols,
                       how='inner',
                       suffixes=(f'_{unique_transpose_values[0]}', f'_{unique_transpose_values[1]}'))
    if len(unique_transpose_values) > 2:
        for unique_val in unique_transpose_values[2:]:
            print(f"Merging {unique_val} into new_df")
            wide_df = wide_df.merge(df_holder_dict[unique_val].add_suffix(f'_{unique_val}'),
                                    left_on=['gene_id', 'gene_name', 'chr_id'],
                                    right_on=[f"{col}_{unique_val}" for col in remerge_cols],
                                    how='inner')
    # print(new_df.columns)
    return wide_df

### Create the bins for genes as they are built in the quickPlotThingsForPolyAVstotalRNA.py:
This is a roundabout way of creating these bins, but it *appears* to be deterministic compared to the way I did it in the other script!

In [7]:
just_2_and_3_df = compressed_df[compressed_df.lib.isin(["polyA2", "polyA3", "totalRNA2", "totalRNA3"])].copy()
just_2_and_3_df = just_2_and_3_df.query("gene_hits >= 80")
bins_2v3_df = multilib_long_to_wide_df(just_2_and_3_df)
for lib_set in [2, 3]:
    bins_2v3_df[f"pA_minus_total_mean_diff_polya_lengths_{lib_set}"] = (bins_2v3_df[f"mean_polya_length_polyA{lib_set}"] -
                                                                        bins_2v3_df[f"mean_polya_length_totalRNA{lib_set}"])
bins_2v3_df[f"avg_diff_of_mean_tail_length_2v3"] = (bins_2v3_df[f"pA_minus_total_mean_diff_polya_lengths_2"] +
                                                    bins_2v3_df[f"pA_minus_total_mean_diff_polya_lengths_3"]) / 2
bins_2v3_df[f'binned_diff_of_mean_tail_length_2v3'], bins = pd.qcut(bins_2v3_df[f'avg_diff_of_mean_tail_length_2v3'], 10,
                                                                    labels=False, retbins=True)
bins_2v3_df[f'binned_diff_of_mean_tail_length_2v3'] += 1

print(bins_2v3_df[['chr_id', 'gene_id', 'gene_name', 'binned_diff_of_mean_tail_length_2v3']])
if 'binned_diff_of_mean_tail_length_2v3' not in compressed_df.columns:
    compressed_df = compressed_df.merge(bins_2v3_df[['gene_id', 'binned_diff_of_mean_tail_length_2v3']], on='gene_id')
    print(compressed_df)

First, merging polyA2 & polyA3
Merging totalRNA2 into new_df
Merging totalRNA3 into new_df
    chr_id         gene_id   gene_name  binned_diff_of_mean_tail_length_2v3
0        I  WBGene00000230       atp-3                                    6
1        I  WBGene00000371      cox-5B                                    1
2        I  WBGene00000625      col-48                                    8
3        I  WBGene00000626      col-49                                    5
4        I  WBGene00000639      col-63                                    2
..     ...             ...         ...                                  ...
584      X  WBGene00020366     acdh-10                                    8
585      X  WBGene00021322    Y34B4A.6                                    1
586      X  WBGene00022415  Y102A11A.5                                    1
587      X  WBGene00022515      ttr-48                                    2
588      X  WBGene00022665     ZK154.1                                   

### Below printing to check against my CSVs I sent to Josh!
#### (Not important for actual run of this, feel free to skip)

In [9]:

col_list = ["gene_id", "gene_name", "avg_diff_of_mean_tail_length_2v3", "binned_diff_of_mean_tail_length_2v3"]
for tail_diff_bin in [10, 9, 8, 7]:
    temp_df = bins_2v3_df.query(f"binned_diff_of_mean_tail_length_2v3 == {tail_diff_bin}")
    temp_df = temp_df[col_list]
    temp_df = temp_df.sort_values('avg_diff_of_mean_tail_length_2v3', ascending=False).reset_index(drop=True)
    print(temp_df)
    # temp_df.to_csv(f"/home/marcus/Documents/220218_bin{bin}genes.tsv", sep="\t", index=False)

           gene_id   gene_name  avg_diff_of_mean_tail_length_2v3  binned_diff_of_mean_tail_length_2v3
0   WBGene00012186      mlt-11                         34.849877                                   10
1   WBGene00020022      gldc-1                         33.610278                                   10
2   WBGene00010988      metr-1                         26.243489                                   10
3   WBGene00019017     F57F4.4                         23.252816                                   10
4   WBGene00006921      vha-12                         21.934854                                   10
5   WBGene00016422      noah-1                         21.901820                                   10
6   WBGene00006912       vha-3                         21.460959                                   10
7   WBGene00009628      tatn-1                         20.472004                                   10
8   WBGene00006727       ubq-1                         19.728632                  

### Do the work of expanding on tail lengths and subsampling each list of polyA tail lengths to get an equal number for each library / gene
*This is definitely an inefficient way to do this... but it was the quickest solution to write for me!*

In [46]:
def subsample_per_gene(compressed_df_with_tails_list, subsampled_tails_per_gene=20,
                       replacement=False) -> (pd.DataFrame, pd.DataFrame):
    from tqdm.notebook import tqdm
    temp_compressed_df = compressed_df_with_tails_list.copy()
    temp_compressed_df["lib_set"] = temp_compressed_df.lib.str[-1]
    temp_compressed_df["lib_type"] = temp_compressed_df.lib.str[:-1]
    exploded_on_tails_df = temp_compressed_df.explode(['polya_lengths', 'read_lengths']).reset_index(drop=True)
    
    new_index_list = ['lib_type', 'lib_set', 'gene_id']
    if new_index_list[0] in exploded_on_tails_df.columns:
        exploded_on_tails_df.set_index(new_index_list, inplace=True)
    exploded_on_tails_df.sort_index(inplace=True)
    exploded_on_tails_df = exploded_on_tails_df.filter(
        ['lib', 'gene_name', 'gene_rpm', 'mean_polya_length', 'polya_lengths', 'binned_diff_of_mean_tail_length_2v3'])
    genes_list = set(exploded_on_tails_df.index.get_level_values('gene_id').to_list())
    genes_iterator = tqdm(genes_list)
    sub_sample_per_gene_lib_set_list = []
    for i, gene in enumerate(genes_iterator):
        genes_iterator.set_description(f"Subsampling to {subsampled_tails_per_gene} tails/gene; working on {gene}")
        for lib_type in ['polyA', 'totalRNA']:
            for lib_set in ['1', '2', '3']:
                if f"{lib_type}{lib_set}" == "totalRNA1":
                    continue
                sub_sample_per_gene_lib_set_list.append(
                    exploded_on_tails_df.loc[(lib_type, lib_set, gene), :].sample(n=subsampled_tails_per_gene, replace=replacement))
    sub_sample_per_gene_df = pd.concat(sub_sample_per_gene_lib_set_list).reset_index()
    return exploded_on_tails_df, sub_sample_per_gene_df


re_expanded_df, sub_sample_per_gene_lib_set_df =  subsample_per_gene(compressed_df,
                                                                     subsampled_tails_per_gene=40,
                                                                     replacement=False)
re_expanded_df.reset_index(inplace=True)
print(sub_sample_per_gene_lib_set_df)
print(re_expanded_df[['lib_type', 'lib_set', 'gene_id', 'gene_rpm', 'polya_lengths', 'binned_diff_of_mean_tail_length_2v3']])

  0%|          | 0/589 [00:00<?, ?it/s]

        lib_type lib_set         gene_id        lib gene_name    gene_rpm  mean_polya_length polya_lengths  binned_diff_of_mean_tail_length_2v3
0          polyA       1  WBGene00019680     polyA1   K12H4.5  336.332029          66.369401         42.28                                    4
1          polyA       1  WBGene00019680     polyA1   K12H4.5  336.332029          66.369401         51.69                                    4
2          polyA       1  WBGene00019680     polyA1   K12H4.5  336.332029          66.369401        225.57                                    4
3          polyA       1  WBGene00019680     polyA1   K12H4.5  336.332029          66.369401         64.32                                    4
4          polyA       1  WBGene00019680     polyA1   K12H4.5  336.332029          66.369401         68.72                                    4
...          ...     ...             ...        ...       ...         ...                ...           ...                              

In [47]:
import plotly.io as pio
pio.renderers.default = "browser"

# This step removes all subsampling...:
# plot_df = re_expanded_df
plot_df = sub_sample_per_gene_lib_set_df.copy()

plot_df = plot_df[plot_df.binned_diff_of_mean_tail_length_2v3.isin([1, 5, 10])]
plot_df.sort_values(['binned_diff_of_mean_tail_length_2v3', 'lib_set'], inplace=True)
plot_df['binned_diff_of_mean_tail_length_2v3'] = plot_df['binned_diff_of_mean_tail_length_2v3'].astype('category')
fig = px.violin(plot_df,
                x='binned_diff_of_mean_tail_length_2v3',
                # facet_row='lib_set',
                color='lib',
                y='polya_lengths',
                log_y=True,
                width=1150,
                height=850,
                range_y=(1, 400),
                labels={'binned_diff_of_mean_tail_length_2v3': 'bin'})
fig.update_traces(meanline_visible=True,
                  # points='all',  # show all points
                  # side='positive',
                  spanmode='hard',
                  # pointpos=-0.1,  # could maybe go back to both sides and zero this...
                  # marker=dict(opacity=0.5),
                  # # jitter=0.05,  # add some jitter on points for better visibility
                  # scalemode='width',  # scale violin plot area with total count
                  )
fig.update_layout(template='plotly_white')
fig.update_xaxes(ticktext=["bin1", "bin5", "bin10"],
                 tickvals=[1,5,10])
fig.show()
#fig.write_image('../testOutputs/220221_deleteme.png')
print('done.')

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_265132/4182640323.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



done.


### Extra analysis to look at if two technical replicates of polyA-selection are at all stratified by the delta-tail-bins

In [36]:
just_pA1_and_pA2_df = compressed_df[compressed_df.lib.isin(["polyA1", "polyA2"])].copy()

# print(just_pA1_and_pA2_df)
just_pA1_and_pA2_df = just_pA1_and_pA2_df.query("gene_hits >= 80")[["lib", "chr_id", "gene_id", "gene_name", "gene_hits", "mean_polya_length", "mean_read_length", "gene_rpm", "binned_diff_of_mean_tail_length_2v3"]]
wide_pA1_and_pA2_df = multilib_long_to_wide_df(just_pA1_and_pA2_df, remerge_cols=("chr_id", "gene_id", "gene_name", "binned_diff_of_mean_tail_length_2v3"))
# print(wide_pA1_and_pA2_df)

wide_pA1_and_pA2_df[f"pA1_over_pA2_rpm"] = (wide_pA1_and_pA2_df[f"gene_rpm_polyA1"] /
                                                            wide_pA1_and_pA2_df[f"gene_rpm_polyA2"])
wide_pA1_and_pA2_df['log2_pA1_over_pA2_rpm'] = np.log2(wide_pA1_and_pA2_df[f"pA1_over_pA2_rpm"])
print(wide_pA1_and_pA2_df[["chr_id", "gene_id", "gene_name", "log2_pA1_over_pA2_rpm", "binned_diff_of_mean_tail_length_2v3"]])
fig = px.ecdf(wide_pA1_and_pA2_df.sort_values(f'binned_diff_of_mean_tail_length_2v3'),
              x=f"log2_pA1_over_pA2_rpm",
              color=f'binned_diff_of_mean_tail_length_2v3',
              color_discrete_sequence=px.colors.sample_colorscale(px.colors.diverging.Portland,
                                                                  [(x/10) for x in range(1, 10+1)]),
              hover_name="gene_name")
fig.update_layout(template='plotly_white')
fig.show()

First, merging polyA1 & polyA2
    chr_id         gene_id   gene_name  log2_pA1_over_pA2_rpm  binned_diff_of_mean_tail_length_2v3
0        I  WBGene00000230       atp-3               0.207258                                    6
1        I  WBGene00000371      cox-5B               0.050617                                    1
2        I  WBGene00000625      col-48               0.189326                                    8
3        I  WBGene00000626      col-49               0.177597                                    5
4        I  WBGene00000639      col-63               0.303883                                    2
..     ...             ...         ...                    ...                                  ...
584      X  WBGene00020366     acdh-10               0.449990                                    8
585      X  WBGene00021322    Y34B4A.6               0.115666                                    1
586      X  WBGene00022415  Y102A11A.5               0.047376                 